## 文档的继承

In [1]:
from mongoengine import connect, disconnect
from mongoengine import Document, DynamicDocument
from mongoengine import StringField, IntField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD

In [2]:
from mongoengine import BooleanField, DateTimeField, DictField, EmailField, FloatField, ListField, ReferenceField

In [3]:
# 连接到已创建的数据库
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)
connect(TEST_DB2, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB2, alias=TEST_DB2)
connect(TEST_DB3, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB3, alias=TEST_DB3)

MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

### 2.3.8. Document inheritance(文档继承)

To create a specialised type of a `Document` you have defined, you may *subclass it and add any extra fields or methods you may need*. As this new class is not a direct subclass of `Document`, **it will not be stored in its own collection**; it will use the same collection as its superclass uses. This allows for more convenient and efficient retrieval of related documents – all you need do is set `allow_inheritance` to `True` in the `meta` data for a document

创建一个继承自现有文档类(此文档类继承自 `Document`)的新类， 新类由于没有直接继承自 `Document`, 所以不会在数据库中建立集合。这样做的好处是**可以在原有的 文档类 基础上添加 额外的字段和类方法**。 需要继承现有文档类的 `meta` 参数 是  `allow_inheritance` to `True`.

In [6]:
class BookInherit(Document):
    bookid = StringField()
    name = StringField()
    meta = {
        "db_alias": TEST_DB1,
        "allow_inheritance": True
        # 不设置此参数时报错: 
        #ValueError: Document BookOrdering may not be subclassed. 
        # To enable inheritance, use the "allow_inheritance" meta attribute.
    }

class BookExtra(BookInherit):
    count = IntField()
    # 继承类，不需要设置 meta 参数

In [31]:
BookInherit(bookid="wx000006", name="水浒传").save()
BookExtra(bookid="wx000007", name="红楼梦", count=109).save()

<BookExtra: BookExtra object>

BookInherit 保存结果:
```json
{
    "_id" : ObjectId("63ff4f123ba566dfccb4e2ac"),
    "_cls" : "BookInherit",
    "bookid" : "wx000006",
    "name" : "水浒传"
}
```
BookExtra 保存结果:
```json
{
    "_id" : ObjectId("63ff4f123ba566dfccb4e2ad"),
    "_cls" : "BookInherit.BookExtra",
    "bookid" : "wx000007",
    "name" : "红楼梦",
    "count" : NumberInt(109)
}
```
可以看到，保存的结果中，**自动添加了字段 `_cls`**

### 继承文档的注意事项

> From 0.8 onwards `allow_inheritance` defaults to `False`, meaning you must set it to `True` to use inheritance. 
> Setting `allow_inheritance` to `True` should also be used in **`EmbeddedDocument`** class in case you need to subclass it

### 继承文档的检索

When it comes to querying using `objects()`, querying `BookInherit.objects()` will query both `BookInherit` and `BookExtra` whereas querying `BookExtra` will only query the `BookExtra` documents. Behind the scenes, MongoEngine deals with inheritance by adding a `_cls` attribute that contains the class name in every documents. When a document is loaded, MongoEngine checks it’s `_cls` attribute and use that class to construct the instance.

即： 基类的检索结果包括基类和子类；而子类的检索结果只包括子类。这个过程，是通过存在继承关系的类文档， 在基类和子类生成的文档中 **自动添加了字段 `_cls`** 完成的。

In [7]:
BookInherit.objects.to_json()

'[{"_id": {"$oid": "63ff4f123ba566dfccb4e2ac"}, "_cls": "BookInherit", "bookid": "wx000006", "name": "\\u6c34\\u6d52\\u4f20"}, {"_id": {"$oid": "63ff4f123ba566dfccb4e2ad"}, "_cls": "BookInherit.BookExtra", "bookid": "wx000007", "name": "\\u7ea2\\u697c\\u68a6", "count": 109}]'

In [8]:
BookExtra.objects.to_json()

'[{"_id": {"$oid": "63ff4f123ba566dfccb4e2ad"}, "_cls": "BookInherit.BookExtra", "bookid": "wx000007", "name": "\\u7ea2\\u697c\\u68a6", "count": 109}]'

### 2.3.8.1 Working with existing data(在已有数据基础上工作)

As MongoEngine no longer defaults to needing `_cls`, you can quickly and easily get working with existing data. Just define the document to match the expected schema in your database  
译: 由于 MongoEngine 不再默认需要 `_cls`，您可以快速轻松地使用现有数据。只需定义文档以匹配数据库中的预期模式。

If you have wildly varying schemas then using a `DynamicDocument` might be more appropriate, instead of defining all possible field types.
译： 如果数据模式变化很大， 在已有数据基础上创建 `DynamicDocument` 动态文档类比定义所有可能的字段更为合适。

If you use `Document` and the database contains data that isn’t defined then that data will be stored in the `document._data` dictionary.
译： 如果你使用了  `Document` 类基础上创建 文档类去匹配现有集合中的文档，没有被定义的已有数据字段，将被储存在 字典 `document._data` 中。


In [4]:
# 使用 DynamicDocument 对应现有的数据库 book_inherit
# 注意参数 "collection": "book_inherit"
class BookExistingData(DynamicDocument):
    bookid = StringField()
    name = StringField()
    meta = {
        "db_alias": TEST_DB1,
        "collection": "book_all_field"
    }

In [5]:
BookExistingData.objects.to_json()
# BookExistingData 中较 数据库中已有文档缺少 count 字段
# 继承了动态文档类之后， 能够 像正常获取文档一样获取数据

'[{"_id": {"$oid": "63fcd8131be34fe2c671fb4b"}, "bookid": "wx092761", "name": "\\u53f2\\u8bb0", "ifnewbook": false, "author": {"$oid": "63fcd8131be34fe2c671fb4a"}, "price": 19.8, "tag": ["\\u5386\\u53f2", "\\u7ecf\\u5178"]}, {"_id": {"$oid": "63fe9de15990799fbfd9fb90"}, "bookid": "wx09sah61", "name": "\\u672c\\u8349\\u7eb2\\u76ee", "ifnewbook": false, "author": {"$oid": "63fe9da75990799fbfd9fb8e"}, "author_email": "lsz@qq.com", "price": 19.8, "book_type": {"$oid": "63fe9da75990799fbfd9fb8f"}, "tag": []}]'

In [6]:
# 使用 Document 对应现有的数据库 book_inherit
# 注意参数 "collection": "book_inherit"
class BookExistingData2(Document):
    bookid = StringField()
    name = StringField()
    meta = {
        "db_alias": TEST_DB1,
        "collection": "book_all_field"
    }

In [7]:
BookExistingData2.objects.to_json()
# BookExistingData 中较 数据库中已有文档缺少 count 字段
# 文档类之后， 能够 像正常获取文档一样获取数据????

'[{"_id": {"$oid": "63fcd8131be34fe2c671fb4b"}, "bookid": "wx092761", "name": "\\u53f2\\u8bb0", "ifnewbook": false, "author": {"$oid": "63fcd8131be34fe2c671fb4a"}, "price": 19.8, "tag": ["\\u5386\\u53f2", "\\u7ecf\\u5178"]}, {"_id": {"$oid": "63fe9de15990799fbfd9fb90"}, "bookid": "wx09sah61", "name": "\\u672c\\u8349\\u7eb2\\u76ee", "ifnewbook": false, "author": {"$oid": "63fe9da75990799fbfd9fb8e"}, "author_email": "lsz@qq.com", "price": 19.8, "book_type": {"$oid": "63fe9da75990799fbfd9fb8f"}, "tag": []}]'

总结: 在现有的数据上 创建 文档类， 有诸多不便， 查询后增加了 新的字段 `"_id": {"$oid": "63fcd8131be34fe2c671fb4b"}`

### 2.3.9. Abstract classes(抽象的文档类)

If you want to add some extra functionality to a group of Document classes but you don’t need or want the overhead of inheritance you can use the `abstract` attribute of `meta`. This won’t turn on Document inheritance but will allow you to keep your code DRY(Don’t repeat yourself).

译： 如果你想为一组文档类(Document classes)添加额外的功能， 但你又不想使用继承的方法，可以使用 创建抽象文档类。这样可以使你的代码保持简洁而不重复(DRY)。

In [14]:
from mongoengine import ValidationError
class BaseDocument(Document):
    meta = {
        "db_alias": TEST_DB1,
        'abstract': True,
    }
    def _not_empty(val):
        if not val:
            # 创建一个通用的，判断输入文本不为空的函数
            raise ValidationError('value can not be empty')

class BookAbstract1(BaseDocument):
    bookid = StringField(validation=BaseDocument._not_empty)
    name = StringField()

In [ ]:
BookAbstract1(bookid="", name="中庸").save()
# 报错如下:
# ValidationError: ValidationError (BookAbstract1:None) (value can not be empty: ['bookid'])

In [17]:
BookAbstract1(bookid="abc", name="中庸").save()  # 正常保存至 集合 book_abstract1 中

<BookAbstract1: BookAbstract1 object>

In [18]:
class BookAbstract2(BaseDocument):
    bookid = StringField()
    name = StringField(validation=BaseDocument._not_empty)

In [ ]:
BookAbstract2(bookid="dshj", name="").save()
# 报错如下:
# ValidationError: ValidationError (BookAbstract2:None) (value can not be empty: ['name'])

In [20]:
BookAbstract2(bookid="dshj", name="诗经").save()  # 正常保存至 集合 book_abstract2 中

<BookAbstract2: BookAbstract2 object>

### 总结

- 2.3.8. Document inheritance(文档继承) 对于扩展文档很有帮助；
- 2.3.9. Abstract classes(抽象的文档类)对于将通用方法写在一起避免重复很有帮助
- 2.3.8.1 Working with existing data(在已有数据基础上工作)，对于更为细致的延续数据管理帮助不大。